# Main app

In [1]:
# Core libraries
import os
import json
import warnings
import threading
from datetime import datetime, timedelta

# Data processing and visualization
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning / Computer Vision
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
import cv2

# GUI
import tkinter as tk
from tkinter import messagebox

# Suppress warnings and TensorFlow logs
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Model paths and configurations
MODEL_PATHS = {
    'glasses': 'dataset_glasses_classification_model.h5',
    'face': 'face_recognition_model.h5',
    'hair': 'hair_detection_model.h5',
    'hair_labels': 'hair_labels.npy'
}

# Detection settings
DETECTION_CONFIG = {
    'cooldown': timedelta(minutes=2),
    'timeout': timedelta(seconds=2),
    'required_detections': 5,
    'confidence_threshold': 0.9
}

# Initialize detection variables
last_detections = {}
detection_buffer = {}
log_file = "detections_log.json"

# Load models
glasses_model = load_model(MODEL_PATHS['glasses'])
face_model = load_model(MODEL_PATHS['face'])
hair_model = load_model(MODEL_PATHS['hair'])
hair_labels = np.load(MODEL_PATHS['hair_labels'])

# Initialize face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Data preparation
data_dir = 'faces'
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_set = datagen.flow_from_directory(
    data_dir, 
    target_size=(180, 180), 
    batch_size=32, 
    class_mode='categorical'
)
identity_labels = {v: k for k, v in train_set.class_indices.items()}

# Initialize video capture
cap = cv2.VideoCapture(1)

# Threading variables
frame_lock = threading.Lock()
current_frame = None
glasses_label = None
predicted_label = None
hair_color_label = None

def train_model():
    data_dir = 'faces'
    batch_size = 32
    img_height, img_width = 180, 180

    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_set = datagen.flow_from_directory(data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical', subset='training')
    test_set = datagen.flow_from_directory(data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical', subset='validation')

    model = models.Sequential()
    model.add(layers.Rescaling(scale=1./255, input_shape=(img_height, img_width, 3)))
    model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(train_set.num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    epochs = 10
    history = model.fit(train_set, validation_data=test_set, epochs=epochs)
    model.save('face_recognition_model.h5')
    print("Model training complete. Saved as 'face_recognition_model.h5'.")

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(epochs)

    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

def advanced_face_detection():
    # Global Variables for detections
    last_detections = {}
    log_file = "detections_log.json"
    detection_cooldown = timedelta(minutes=2)
    detection_buffer = {}
    required_consistent_detections = 5
    detection_timeout = timedelta(seconds=2)
    confidence_threshold = 0.9

    # Load Models
    try:
        glasses_model = load_model('dataset_glasses_classification_model.h5')
        face_model = load_model('face_recognition_model.h5')
        hair_model = load_model('hair_detection_model.h5')
        hair_labels = np.load('hair_labels.npy')
    except Exception as e:
        messagebox.showerror("Error", f"Erreur lors du chargement des modèles: {str(e)}")
        return

    # Variables for synchronisation
    frame_lock = threading.Lock()
    current_frame = None
    glasses_label = None
    predicted_label = None
    hair_color_label = None
    last_recognized_person = ""


    def write_detection_log(identity, glasses_status, hair_color):
        current_time = datetime.now()
        if identity in last_detections:
            if (current_time - last_detections[identity]) < detection_cooldown:
                return
        
        last_detections[identity] = current_time
        log_entry = {
            "timestamp": current_time.strftime("%Y-%m-%d %H:%M:%S"),
            "identity": identity,
            "glasses": glasses_status,
            "hair_color": hair_color
        }
        
        try:
            try:
                with open(log_file, 'r') as f:
                    logs = json.load(f)
            except FileNotFoundError:
                logs = []
            logs.append(log_entry)
            with open(log_file, 'w') as f:
                json.dump(logs, f, indent=4)
        except Exception as e:
            print(f"Erreur lors de l'écriture des logs: {e}")

    def detect_glasses_and_identity():
        nonlocal glasses_label, predicted_label, last_recognized_person
        while True:
            with frame_lock:
                if current_frame is None:
                    continue
                frame = current_frame.copy()

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=7, minSize=(50, 50))

            for (x, y, w, h) in faces:
                face = frame[y:y+h, x:x+w]

                # Glasses detections
                resized_face_glasses = cv2.resize(face, (128, 128))
                normalized_face_glasses = resized_face_glasses / 255.0
                input_face_glasses = np.expand_dims(normalized_face_glasses, axis=0)
                glasses_prediction = glasses_model.predict(input_face_glasses, verbose=0)
                glasses_label = "Glasses" if glasses_prediction[0] <= 0.5 else "No Glasses"

                # Identity recognition
                resized_face_identity = cv2.resize(face, (180, 180))
                normalized_face_identity = resized_face_identity / 255.0
                input_face_identity = np.expand_dims(normalized_face_identity, axis=0)
                identity_predictions = face_model.predict(input_face_identity, verbose=0)
                
                confidence = np.max(identity_predictions[0])
                predicted_index = np.argmax(identity_predictions[0])
                
                if confidence >= confidence_threshold:
                    current_identity = identity_labels[predicted_index]
                    current_time = datetime.now()
                    
                    if current_identity in detection_buffer:
                        detection_buffer[current_identity] = [
                            (time, ident, glasses, hair) 
                            for time, ident, glasses, hair in detection_buffer[current_identity] 
                            if current_time - time < detection_timeout
                        ]
                    
                    if current_identity not in detection_buffer:
                        detection_buffer[current_identity] = []
                    detection_buffer[current_identity].append(
                        (current_time, current_identity, glasses_label, hair_color_label)
                    )
                    
                    if len(detection_buffer[current_identity]) >= required_consistent_detections:
                        predicted_label = current_identity
                        last_recognized_person = current_identity
                        write_detection_log(
                            identity=current_identity,
                            glasses_status=glasses_label,
                            hair_color=hair_color_label
                        )
                        detection_buffer[current_identity] = []
                    else:
                        predicted_label = "Verification..."
                else:
                    predicted_label = "Inconnu"

    def detect_hair_color():
        nonlocal hair_color_label
        while True:
            with frame_lock:
                if current_frame is None:
                    continue
                frame = current_frame.copy()

            resized_frame = cv2.resize(frame, (224, 224))
            input_frame = preprocess_input(np.expand_dims(resized_frame, axis=0))
            predictions = hair_model.predict(input_frame, verbose=0)
            hair_color_label = hair_labels[np.argmax(predictions)]

    # Start threads
    glasses_and_identity_thread = threading.Thread(target=detect_glasses_and_identity, daemon=True)
    hair_color_thread = threading.Thread(target=detect_hair_color, daemon=True)

    glasses_and_identity_thread.start()
    hair_color_thread.start()

    # Video Capture
    cap = cv2.VideoCapture(1)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        with frame_lock:
            current_frame = frame.copy()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=7, minSize=(50, 50))

        cv2.putText(frame, f"Last Person Recognised: {last_recognized_person}", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            labels = [predicted_label, glasses_label, hair_color_label]
            for i, label in enumerate(labels):
                if label:  # Vérifier que le label n'est pas None
                    cv2.putText(frame, str(label), (x, y - 10 - (i * 30)), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Reconnaissance Faciale", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def add_face():
    def start_capture():
        name = name_entry.get().strip()
        if not name:
            messagebox.showerror("Error", "Name cannot be empty!")
            return
        
        person_folder = os.path.join('faces', name)
        if os.path.exists(person_folder):
            message = f"Adding pictures to the existing folder for {name}."
        else:
            message = f"Creating a new folder for {name}."
            os.makedirs(person_folder, exist_ok=True)

        capture_window.destroy()  # Close the name entry window

        cap = cv2.VideoCapture(1)

        # Display the message on the camera feed
        for i in range(3, 0, -1):
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.putText(frame, message, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            frame = cv2.putText(frame, f"Capturing in {i}...", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow("Add New Face", frame)
            cv2.waitKey(1000)  # Wait for 1 second per countdown step

        print("Press 'SPACE' to stop capturing photos.")
        count = len(os.listdir(person_folder))  # Start from the next available file number

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(
                gray, scaleFactor=1.1, minNeighbors=7, minSize=(50, 50)
            )  # Increased minNeighbors and added minSize
            for (x, y, w, h) in faces:
                if w < 50 or h < 50:  # Skip faces that are too small
                    continue
                face = frame[y:y+h, x:x+w]
                face_path = os.path.join(person_folder, f"{name}_{count}.jpg")
                cv2.imwrite(face_path, face)
                count += 1
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            frame = cv2.putText(frame, "Press 'SPACE' to stop", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow("Add New Face", frame)

            key = cv2.waitKey(1)
            if key == 32:  # Spacebar
                break

        cap.release()
        cv2.destroyAllWindows()
        print(f"Saved {count - len(os.listdir(person_folder))} new photos for {name}.")

    # Create a new window to input the name
    capture_window = tk.Tk()
    capture_window.title("Add a New Person")
    capture_window.geometry("300x200")

    tk.Label(capture_window, text="Enter Name:", font=("Helvetica", 12)).pack(pady=10)
    name_entry = tk.Entry(capture_window, font=("Helvetica", 12))
    name_entry.pack(pady=10)

    tk.Button(capture_window, text="Start Capture", font=("Helvetica", 12), command=start_capture).pack(pady=20)
    capture_window.mainloop()
    
# Main Menu
def open_main_menu():
    root = tk.Tk()
    root.title("Face Recognition App")
    root.geometry("400x400")

    tk.Label(root, text="Face Recognition App", font=("Helvetica", 16)).pack(pady=20)
    tk.Button(root, text="Add a New Person", font=("Helvetica", 14), command=lambda: [root.destroy(), add_face()]).pack(pady=20)
    tk.Button(root, text="Start Advanced Detection", font=("Helvetica", 14), command=lambda: [root.destroy(), advanced_face_detection()]).pack(pady=20)
    tk.Button(root, text="Train Model", font=("Helvetica", 14), command=lambda: [root.destroy(), train_model()]).pack(pady=20)
    tk.Button(root, text="Exit", font=("Helvetica", 14), command=root.quit).pack(pady=20)

    root.mainloop()

# Run the menu
if __name__ == "__main__":
    open_main_menu()

/Users/antoine/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Found 4639 images belonging to 21 classes.


2025-01-15 10:05:58.351 Python[45277:11735074] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-01-15 10:05:59.676 Python[45277:11735074] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


# Second App

Found 2604 images belonging to 21 classes.
Found 641 images belonging to 21 classes.
Number of classes: 21
Training batch shape: (32, 180, 180, 3) (32, 21)
Validation batch shape: (32, 180, 180, 3) (32, 21)
Epoch 1/10
42/82 ━━━━━━━━━━━━━━━━━━━━ 5s 142ms/step - accuracy: 0.1039 - loss: 3.0019

: 